In [340]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from shapely.geometry import MultiPolygon
import pandas as pd

In [247]:
!ls

2010demo.json                      VA_demo2010.json
README.rtf                         VA_demo_Filtered.json
RI_demo2010.json                   all_states_precincts_neighbor.json
RI_demo_Filtered.json              demoBreakDown.ipynb
TX_demo2010.json                   demoMapper.ipynb
TX_demo_Filtered.json              merged_unclean_demo.json


In [248]:
precinct = gpd.read_file("all_states_precincts_neighbor.json")
precinct.head(1)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,Neighbors,geometry
0,Anderson,1,262,742,18,5,3,1030,tx,tx-anderson-1,"tx-anderson-1,tx-anderson-3; tx-anderson-1,tx-...","POLYGON ((-95.60102 31.74519, -95.60095 31.745..."


In [249]:
precinctCopy = precinct.copy()

# Filtering out demo polygons by stateFIPs
    - RI: 44
    - VA: 51
    - TX: 48

In [251]:
def filterPolygon(demoDF, stateFIPS):
    for index, row in demoDF.iterrows():
        cur_stateFIPS = row["GEOID10"][:2]
        if cur_stateFIPS != stateFIPS:
            demoDF.drop(index, inplace=True)
            print("@",end="")
    return demoDF

In [256]:
RIdemo = gpd.read_file("RI_demo2010.json")
RIdemo_Filtereed = filterPolygon(RIdemo, "44")
# drop belongsTo
RIdemo_Filtereed.drop(["belongsTo"], axis=1, inplace=True)
RIdemo_Filtereed.to_file("RI_demo_Filtered.json", driver="GeoJSON")

@@@@@@@@@@@@@@@@@@@@@@@@@@

In [258]:
VAdemo = gpd.read_file("VA_demo2010.json")
VAdemo_Filtereed = filterPolygon(VAdemo, "51")
# drop belongsTo
VAdemo_Filtereed.drop(["belongsTo"], axis=1, inplace=True)
VAdemo_Filtereed.to_file("VA_demo_Filtered.json", driver="GeoJSON")

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [259]:
TXdemo = gpd.read_file("TX_demo2010.json")
TXdemo_Filtereed = filterPolygon(TXdemo, "48")
# drop belongsTo
TXdemo_Filtereed.drop(["belongsTo"], axis=1, inplace=True)
TXdemo_Filtereed.to_file("TX_demo_Filtered.json", driver="GeoJSON")

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [260]:
RIdemoCopy = RIdemo_Filtereed.copy()
VAdemoCopy = VAdemo_Filtereed.copy()
TXdemoCopy = TXdemo_Filtereed.copy()

# Constants

# Starting mapping demo to RI Precincts

In [329]:
def addCols(df):
    cols = ["Total", "White", "Black or African American", "American Indian and Alaska Native", "Asian", "Other"]
    for c in cols:
        df[c] = 0
    return df

def unpackDemo(demoRow):
    d = {}
    d["Total"] = demoRow["Total:"]
    d["White"] = demoRow["White"]
    d["Black or African American"] = demoRow["Black or African American"]
    d["American Indian and Alaska Native"] = demoRow["American Indian and Alaska Native"]
    d["Asian"] = demoRow["Asian:"]
    d["Other"] = demoRow["Other"]
    return d

def percentOverlap(demoPolygon, intersectingPolygons):
    d = {}
    demoPolyArea = demoPolygon.area
    for index, row in intersectingPolygons.iterrows():
        intersectionPoly = demoPolygon.intersection(row["geometry"])
        d[index] = intersectionPoly.area / demoPolyArea
        
    # if overlap percentage is less than 5%, ignore it
    new_d = {}
    for key, value in d.items():
        if value > 0.05:
            new_d[key] = value
            
    return new_d

def getDemoByPercent(demoUnpacked, percent):
    d = {}
    for key, value in demoUnpacked.items():
        d[key] = int(value*percent)
    return d

def fillUpPercent(indexPercOverlapMap):
    total = 0
    for index, percent in indexPercOverlapMap.items():
        total += percent

    for index, percent in indexPercOverlapMap.items():
        indexPercOverlapMap[index] = percent/total
  
    return indexPercOverlapMap

def splitByPercOverlap(indexPercOverlapMap, demoUnpacked):
    d = {}
    for index, percent in indexPercOverlapMap.items():
        d[index] = getDemoByPercent(demoUnpacked, percent)
        
    return d

def fillPrecinctDemo(precinctDF, distributeDemo):
    for index, demoMap in distributeDemo.items():
        for colName, demoValue in demoMap.items():
            precinctDF.at[index, colName] += demoValue

In [330]:
RI_Precinct = precinct[precinct["State"] == "ri"]
RIdemo_Filtereed = RIdemoCopy.copy()

In [331]:
# start mapping the demo data to RI
RI_Precinct = addCols(RI_Precinct)
counter = 0
for index, row in RIdemo_Filtereed.iterrows():
    if counter%100 == 0:
        print("#", end="")
        
    cur_demo_geometry = row["geometry"]
    intersectingPolygons = RI_Precinct[RI_Precinct["geometry"].intersects(cur_demo_geometry)]
    demoUnpacked = unpackDemo(row)
    indexPercOverlapMap = percentOverlap(cur_demo_geometry, intersectingPolygons)
    indexPercOverlapMap = fillUpPercent(indexPercOverlapMap)
    distributeDemo = splitByPercOverlap(indexPercOverlapMap, demoUnpacked)
    fillPrecinctDemo(RI_Precinct, distributeDemo)
    
    counter += 1

###

In [332]:
RI_Precinct.head(3)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,Neighbors,geometry,Total,White,Black or African American,American Indian and Alaska Native,Asian,Other
9082,Bristol,0101,1027,522,60,23,2,1634,ri,ri-bristol-0101,"ri-bristol-0101,ri-bristol-0102; ri-bristol-01...","POLYGON ((-71.32270 41.77270, -71.32270 41.772...",3268,3055,23,3,110,73
9083,Bristol,0102,1331,683,68,21,3,2106,ri,ri-bristol-0102,"ri-bristol-0102,ri-bristol-0101; ri-bristol-01...","MULTIPOLYGON (((-71.35581 41.74533, -71.35560 ...",2921,2740,18,3,85,70
9084,Bristol,0103,1136,469,59,17,0,1681,ri,ri-bristol-0103,"ri-bristol-0103,ri-bristol-0102; ri-bristol-01...","POLYGON ((-71.31979 41.73751, -71.31897 41.737...",3544,3395,8,7,64,64


# mapping demo to VA

In [333]:
VA_Precinct = precinct[precinct["State"] == "va"]
VAdemo_Filtereed = VAdemoCopy.copy()

In [334]:
# fix self-intersecting polygon
def fixSelfInter(gdf):
    gdf["new_g"] = ""
    for index, row in gdf.iterrows():
        cur_g = row["geometry"]
        if cur_g.geom_type == "MultiPolygon":
            polys = []
            for shape in cur_g:
                if not shape.is_valid:
                    polys.append(shape.buffer(0.0))
                else:
                    polys.append(shape)
            multiPoly = MultiPolygon(polys)
            gdf.at[index, "new_g"] = multiPoly
        else:
            if not cur_g.is_valid:
                cur_g = cur_g.buffer(0.0)
            gdf.at[index, "new_g"] = cur_g
    gdf.set_geometry("new_g", drop=True, inplace=True)
    return gdf

In [335]:
# start mapping the demo data to VA
VA_Precinct = addCols(VA_Precinct)
VA_Precinct = fixSelfInter(VA_Precinct)
counter = 0
for index, row in VAdemo_Filtereed.iterrows():
    if counter%100 == 0:
        print("#", end="")
        
    cur_demo_geometry = row["geometry"]
    intersectingPolygons = VA_Precinct[VA_Precinct["geometry"].intersects(cur_demo_geometry)]
    demoUnpacked = unpackDemo(row)
    indexPercOverlapMap = percentOverlap(cur_demo_geometry, intersectingPolygons)
    indexPercOverlapMap = fillUpPercent(indexPercOverlapMap)
    distributeDemo = splitByPercOverlap(indexPercOverlapMap, demoUnpacked)
    fillPrecinctDemo(VA_Precinct, distributeDemo)
    
    counter += 1

####################

In [336]:
VA_Precinct.head(3)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,Neighbors,geometry,Total,White,Black or African American,American Indian and Alaska Native,Asian,Other
9501,Accomack,0001,486,637,16,8,4,2328,va,va-accomack-0001,"va-accomack-0001,va-accomack-0010; va-accomack...","POLYGON ((-75.66233 37.77053, -75.66244 37.770...",3526,2336,906,12,20,246
9502,Accomack,0002,175,553,13,1,1,1500,va,va-accomack-0002,"va-accomack-0002,va-accomack-0003; va-accomack...","POLYGON ((-75.41651 37.93483, -75.41645 37.934...",1522,900,492,4,7,112
9503,Accomack,0003,141,342,9,1,0,992,va,va-accomack-0003,"va-accomack-0003,va-accomack-0002; va-accomack...","POLYGON ((-75.59939 37.87382, -75.59898 37.873...",546,405,103,0,2,33


# Mapping for TX

In [337]:
TX_Precinct = precinct[precinct["State"] == "tx"]
TXdemo_Filtereed = TXdemoCopy.copy()

In [338]:
# start mapping the demo data to TX
TX_Precinct = addCols(TX_Precinct)
TX_Precinct = fixSelfInter(TX_Precinct)
counter = 0
for index, row in TXdemo_Filtereed.iterrows():
    if counter%100 == 0:
        print("#", end="")
        
    cur_demo_geometry = row["geometry"]
    intersectingPolygons = TX_Precinct[TX_Precinct["geometry"].intersects(cur_demo_geometry)]
    demoUnpacked = unpackDemo(row)
    indexPercOverlapMap = percentOverlap(cur_demo_geometry, intersectingPolygons)
    indexPercOverlapMap = fillUpPercent(indexPercOverlapMap)
    distributeDemo = splitByPercOverlap(indexPercOverlapMap, demoUnpacked)
    fillPrecinctDemo(TX_Precinct, distributeDemo)
    
    counter += 1

#####################################################

In [339]:
TX_Precinct.head(3)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,Neighbors,geometry,Total,White,Black or African American,American Indian and Alaska Native,Asian,Other
0,Anderson,1,262,742,18,5,3,1030,tx,tx-anderson-1,"tx-anderson-1,tx-anderson-3; tx-anderson-1,tx-...","POLYGON ((-95.60102 31.74519, -95.60095 31.745...",2651,1965,393,16,37,239
1,Anderson,2,181,1318,22,5,6,1532,tx,tx-anderson-2,"tx-anderson-2,tx-anderson-3; tx-anderson-2,tx-...","POLYGON ((-95.55618 31.68776, -95.55616 31.687...",3095,2810,146,24,6,107
2,Anderson,3,94,663,16,2,1,776,tx,tx-anderson-3,"tx-anderson-3,tx-anderson-1; tx-anderson-3,tx-...","POLYGON ((-95.49198 31.74147, -95.49208 31.741...",2133,1623,289,11,21,184


# merge all 3 state with demo

In [341]:
merged = pd.concat([RI_Precinct, VA_Precinct, TX_Precinct], ignore_index=True)
merged.sort_values(["State", "County", "CName"], inplace=True)
merged.head()

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,Neighbors,geometry,Total,White,Black or African American,American Indian and Alaska Native,Asian,Other
0,Bristol,0101,1027,522,60,23,2,1634,ri,ri-bristol-0101,"ri-bristol-0101,ri-bristol-0102; ri-bristol-01...","POLYGON ((-71.32270 41.77270, -71.32270 41.772...",3268,3055,23,3,110,73
1,Bristol,0102,1331,683,68,21,3,2106,ri,ri-bristol-0102,"ri-bristol-0102,ri-bristol-0101; ri-bristol-01...","MULTIPOLYGON (((-71.35581 41.74533, -71.35560 ...",2921,2740,18,3,85,70
2,Bristol,0103,1136,469,59,17,0,1681,ri,ri-bristol-0103,"ri-bristol-0103,ri-bristol-0102; ri-bristol-01...","POLYGON ((-71.31979 41.73751, -71.31897 41.737...",3544,3395,8,7,64,64
3,Bristol,0104,1328,611,78,25,2,2044,ri,ri-bristol-0104,"ri-bristol-0104,ri-bristol-0101; ri-bristol-01...","MULTIPOLYGON (((-71.30677 41.74204, -71.30682 ...",3416,3236,16,0,110,49
4,Bristol,0105,1290,590,69,24,4,1977,ri,ri-bristol-0105,"ri-bristol-0105,ri-bristol-0103; ri-bristol-01...","MULTIPOLYGON (((-71.31354 41.72381, -71.31370 ...",3156,3017,8,3,75,47


In [342]:
merged.to_file("all_states_precincts_cname_neighbor_demo.json", driver="GeoJSON")